In [1]:
from tqdm import tqdm
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from xgboost import XGBClassifier 

## load normalized text

In [2]:
data_all = pd.read_csv('all_docs.csv')
data_all = data_all.replace(np.nan, 'noinformation') # have some empty text(the problem had to solve by Sergey)

## load train and text

In [3]:
tr = pd.read_csv('kirill/train_groups.csv')
ts  = pd.read_csv('kirill/test_groups.csv')

ts['is_train'] = False
tr['is_train'] = True

data_x = pd.concat(( tr.drop('target', 1), ts))

title = pd.read_csv('kirill/information.csv')
satistic = data_x.merge(title, how = 'left', on = 'doc_id')
satistic = satistic.replace(np.nan, 'noinformation')

numbers = []
num = np.arange(10).astype(str)
for i in satistic.title:
    numbers.append(sum(letter in num for letter in i))
    
    
satistic['numbers'] = numbers


vowels = []
vowel = ['а', 'о', 'и', 'е', 'ё', 'э', 'ы', 'у', 'ю', 'я']
for i in satistic.title:
    vowels.append(sum(letter in vowel for letter in i))
satistic['vowels'] = vowels


consonants = []
consonant = ['б', 'в', 'г', 'д', 'ж', 'з', 'й', 'к', 'л', 'м', 'н', 'п', 'р','с', 'т', 'ф', 'х', 'ц', 'ч', 'ш', 'щ']
for i in satistic.title:
    consonants.append(sum(letter in consonant for letter in i))
satistic['consonants'] = consonants

In [4]:
train_st = satistic[satistic.is_train]
test_st = satistic[~satistic.is_train]

In [5]:
data_all = satistic.merge(data_all, how = 'left', on = 'doc_id')

In [21]:
vectorizer = TfidfVectorizer(max_df = 0.8, min_df = 0.2,
dtype = np.float32,
use_idf=True,
ngram_range=(1,1))
data_tf_1 = vectorizer.fit_transform(data_all.text)
data_tf = hstack([data_tf, data_tf_1])

NameError: name 'fata_tf_1' is not defined

## Копипаст Севы

In [23]:
doc_to_title = {}
with open('titles_norm_ru.csv', encoding='utf-8', newline='') as f:
    for num_line, line in enumerate(f):
        if num_line == 0:
            continue
        data = line.strip().split('\t', 1)
        
        doc_id = int(data[0])
        if len(data) == 1:
            title = ''
        else:
            title = data[1]
        doc_to_title[doc_id] = title
print (len(doc_to_title))

28026


In [24]:
train_data = pd.read_csv('kirill/train_groups.csv')
traingroups_titledata = {}
for i in range(len(train_data)):
    new_doc = train_data.iloc[i]
    doc_group = new_doc['group_id']
    doc_id = new_doc['doc_id']
    target = new_doc['target']
    title = doc_to_title[doc_id]
    if doc_group not in traingroups_titledata:
        traingroups_titledata[doc_group] = []
    traingroups_titledata[doc_group].append((doc_id, title, target))

In [25]:
y_train = []
X_train = []
groups_train = []
for new_group in traingroups_titledata:
    docs = traingroups_titledata[new_group]
    for k, (doc_id, title, target_id) in enumerate(docs):
        y_train.append(target_id)
        groups_train.append(new_group)
        all_dist = []
        words = set(title.strip().split())
        for j in range(0, len(docs)):
            if k == j:
                continue
            doc_id_j, title_j, target_j = docs[j]
            words_j = set(title_j.strip().split())
            all_dist.append(len(words.intersection(words_j)))
        X_train.append(sorted(all_dist, reverse=True)[0:19]    )
X_train = np.column_stack((X_train, train_st[['numbers','vowels', 'consonants']]))
# X_train = np.column_stack((X_train, new_train[['mean_true_hash']]))
X_train = np.array(X_train)
y_train = np.array(y_train)
groups_train = np.array(groups_train)
print (X_train.shape, y_train.shape, groups_train.shape)

(11690, 22) (11690,) (11690,)


In [26]:
test_data = pd.read_csv('kirill/test_groups.csv')
testgroups_titledata = {}
for i in range(len(test_data)):
    new_doc = test_data.iloc[i]
    doc_group = new_doc['group_id']
    doc_id = new_doc['doc_id']
    title = doc_to_title[doc_id]
    if doc_group not in testgroups_titledata:
        testgroups_titledata[doc_group] = []
    testgroups_titledata[doc_group].append((doc_id, title))

In [27]:
X_test = []
groups_test = []
for new_group in testgroups_titledata:
    docs = testgroups_titledata[new_group]
    for k, (doc_id, title) in enumerate(docs):
        groups_test.append(new_group)
        all_dist = []
        words = set(title.strip().split())
        for j in range(0, len(docs)):
            if k == j:
                continue
            doc_id_j, title_j = docs[j]
            words_j = set(title_j.strip().split())
            all_dist.append(len(words.intersection(words_j)))
        X_test.append(sorted(all_dist, reverse=True)[0:19]    )
X_test = np.column_stack((X_test, test_st[['numbers','vowels', 'consonants']]))
# X_test = np.column_stack((X_test, new_test[['mean_true_hash']]))
X_test = np.array(X_test)
groups_test = np.array(groups_test)
print (X_test.shape, groups_test.shape)

(16627, 22) (16627,)


## Make TfIdf of all text

In [28]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer

model = Pipeline([
    ('tf', TfidfVectorizer()),
    ('clf', XGBClassifier())])

In [29]:
from itertools import product
from sklearn.metrics import f1_score
from scipy.sparse import hstack

In [30]:
data_tf.shape

(28317, 2129)

In [33]:
groups = train_data.group_id.unique()
length = len(train_data[train_data['group_id'].isin(groups[:int(len(groups)*4/6)])])

train_tf = data_tf.tocsr()[:len(train_data)]
test_tf = data_tf.tocsr()[len(train_data):]

train_new = hstack([train_tf, X_train])
test_new = hstack([test_tf, X_test])
    
X = train_new.tocsc()[:length]
X_val = train_new.tocsc()[length:]
y = y_train[:length]
y_val = y_train[length:]
model = XGBClassifier(max_depth = 4, learning_rate = 0.01, 
                        random_state = 45, n_estimators  = 500, n_jobs = -1)
    
model.fit(X, y)
y_pred = model.predict(X_val)

print(f1_score(y_val, y_pred))

0.674916706330319


In [ ]:
model.fit(train_new, y_train)
pred_xgb = model.predict(test_new)
from collections import Counter
Counter(pred_xgb)

In [ ]:
nums = list(np.arange(11691, 28318))

df = pd.DataFrame({'pair_id': nums, 'target': pred_xgb})
df.to_csv('xgb_text.csv', index=False)